#### Step 1: Prepare documents

In [3]:
!head documents.json

[
  {
    "course": "data-engineering-zoomcamp",
    "documents": [
      {
        "text": "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  \u201cOffice Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon\u2019t forget to register in DataTalks.Club's Slack and join the channel.",
        "section": "General course-related questions",
        "question": "Course - When will the course start?"
      },
      {


In [2]:
!head paraphrase-albert-small-v2_based_embeddings.json

[
  {
    "text": "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  \u201cOffice Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon\u2019t forget to register in DataTalks.Club's Slack and join the channel.",
    "section": "General course-related questions",
    "question": "Course - When will the course start?",
    "course": "data-engineering-zoomcamp",
    "text_vector": [
      0.1563597321510315,
      0.22758997976779938,
      -0.6082090139389038,


In [26]:
import json

with open('documents.json') as f:
  camps = json.load(f)

In [27]:
documents = []

for camp in camps:
  course_name = camp['course']
  for doc in camp['documents']:
    doc['course'] = course_name
    documents.append(doc)

documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

#### step 1.1 Loading prev embeddings

In [5]:
import json

with open('paraphrase-albert-small-v2_based_embeddings.json') as f:
  documents = json.load(f)

len(documents)

948

#### Step 2: Create Embeddings using Pretrained Models

In [11]:
from sentence_transformers import SentenceTransformer

In [12]:
transformer = SentenceTransformer('paraphrase-albert-small-v2')

In [22]:
transformer.encode("hhhh")

array([ 9.58226383e-01,  2.36796483e-01, -8.77282798e-01, -5.96352994e-01,
       -4.54646021e-01,  2.89680094e-01,  6.81344151e-01, -2.30856702e-01,
       -7.94344366e-01,  1.07218876e-01,  9.20639783e-02,  8.62520695e-01,
       -3.52899790e-01,  5.04674733e-01, -3.62859219e-01,  6.54436767e-01,
       -9.45782661e-01, -3.18103790e-01,  6.91944301e-01,  1.16856730e+00,
        8.04190040e-02,  7.91758060e-01,  8.78863275e-01, -3.94959301e-01,
       -5.46624124e-01, -1.92195237e-01,  1.08990705e+00,  3.47614795e-01,
       -5.16024828e-01,  1.32297665e-01,  6.21051073e-01,  4.73846972e-01,
        8.53618622e-01, -2.07517460e-01, -1.01165688e+00, -7.26241708e-01,
       -1.55185795e+00, -1.14049735e-02, -1.20190084e-01, -9.53526318e-01,
        9.22738075e-01, -7.62222558e-02, -4.13750142e-01,  6.07849956e-02,
        5.06582022e-01, -2.96173245e-01, -5.94703078e-01, -5.31909943e-01,
        7.02299923e-02, -2.08733678e-02,  1.71513230e-01,  2.12623402e-01,
       -2.02030942e-01,  

In [23]:
# length of the embedding vector
len(_)

768

In [28]:
for document in documents:
  document['text_vector'] = transformer.encode(document['text']).tolist()

In [29]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'text_vector': [0.1563597321510315,
  0.22758997976779938,
  -0.6082090139389038,
  -0.18197789788246155,
  -0.0659043937921524,
  0.05164671689271927,
  0.5170499086380005,
  0.47857749462127686,
  -0.17045816779136658,
  -0.0892346203327179,
  -0.10816008597612381,
  -0.29102015495300293,
  0.147953063249588,
  0.2420869916677475,
  0.07755620032548904,
  -0.0900619700551033,


In [34]:
with open('paraphrase-albert-small-v2_based_embeddings.json', 'w') as result:
  json.dump(documents, result, indent=2)

Sentence Transformers documentation here: https://www.sbert.net/docs/sentence_transformer/pretrained_models.html

#### Step 3: Setup ElasticSearch connection 

In [6]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

es_client.info()

ObjectApiResponse({'name': 'a2b96b4ec4d2', 'cluster_name': 'docker-cluster', 'cluster_uuid': '7A7xaY3ZSfqz97zYNQ7GdQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

#### Step 4: Create Mappings and Index

* Mapping is the process of defining how a document, and the fields it contains, are stored and indexed.

* Each document is a collection of fields, which each have their own data type.

* We can compare mapping to a database schema in how it describes the fields and properties that documents hold, the datatype of each field (e.g., string, integer, or date), and how those fields should be indexed and stored 



In [7]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "text_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
        }
    }
}

In [8]:
index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

#### Step 5: Add documents into index

In [9]:
for doc in documents:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

#### Step 6: Create end user query

In [13]:
search_term = "windows or mac?"
vector_search_term = transformer.encode(search_term)

In [26]:
knn_query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 2,
    "num_candidates": 10000, 
}

In [27]:
res = es_client.search(
  index=index_name, 
  knn=knn_query,
  source=["text", "course"],
  min_score=0.6
)
res["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'vi3UtZAB9gCCBRV552N0',
  '_score': 0.75794566,
  '_source': {'course': 'machine-learning-zoomcamp',
   'text': 'Refer to the page https://docs.docker.com/desktop/install/mac-install/ remember to check if you have apple chip or intel chip.'}},
 {'_index': 'course-questions',
  '_id': 'ey3UtZAB9gCCBRV58WQB',
  '_score': 0.6514921,
  '_source': {'course': 'machine-learning-zoomcamp',
   'text': "Q: Hii folks, I tried deploying my docker image on Render, but it won't I get SIGTERM everytime.\nI think .5GB RAM is not enough, is there any other free alternative available ?\nA: aws (amazon), gcp (google), saturn.\nBoth aws and gcp give microinstance for free for a VERY long time, and a bunch more free stuff.\nSaturn even provides free GPU instances. Recent promo link from mlzoomcamp for Saturn:\n“You can sign up here: https://bit.ly/saturn-mlzoomcamp\nWhen you sign up, write in the chat box that you're an ML Zoomcamp student and you should get extra G

#### Step 7: Perform Keyword search with Semantic Search (Hybrid/Advanced Search)

In [28]:
result = es_client.search(
  index=index_name,
  # stricting the results come only from one course
  knn=knn_query, # semantic search
  query={"match": {"course": "data-engineering-zoomcamp"}}, # keyword search
  size=5,
  source=['course', 'question', 'text'],
  explain=True
)

In [29]:
result['hits']['hits']

[{'_shard': '[course-questions][0]',
  '_node': 'Ux4ozs4mQimXvy4jQMSYnA',
  '_index': 'course-questions',
  '_id': 'Zi3UtZAB9gCCBRV5w2HH',
  '_score': 0.778914,
  '_source': {'question': 'Course - When will the course start?',
   'course': 'data-engineering-zoomcamp',
   'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel."},
  '_explanation': {'value': 0.778914,
   'description': 'sum of:',
   'details': [{'value': 0.778914,
     'description': 'weight(course:data-engineering-zoomcamp in 0) [PerFieldSimilarity], result of:',
     'details': [{'value': 0.778914,
       'de

#### References

1. https://logz.io/blog/elasticsearch-mapping/#:~:text=Within%20a%20search%20engine%2C%20mapping,indexes%20and%20stores%20its%20fields

2. https://www.sbert.net/docs/sentence_transformer/pretrained_models.html

3. https://www.elastic.co/search-labs/tutorials

4. https://www.elastic.co/search-labs/blog/text-similarity-search-with-vectors-in-elasticsearch